In [ ]:
%matplotlib inline 

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

import model
import model_utils
import preprocessing_utils

import optuna

In [ ]:
torch.manual_seed(1234)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs = 500
patience = 5

num_timesteps_input = 7
num_timesteps_output = 4

In [ ]:
raw_trunc_dir = "./data/raw/trunc/"
process_dir = "./data/processed/"

preprocessing_utils.processed(raw_trunc_dir, process_dir, overwrite=False)
A, X, metadata, cat2index, timestamps, means, stds = preprocessing_utils.load(process_dir)

split_line1 = int(X.shape[2] * 0.6)
split_line2 = int(X.shape[2] * 0.8)

train_original_data = X[:, :, :split_line1]
val_original_data = X[:, :, split_line1:split_line2]
test_original_data = X[:, :, split_line2:]

training_input, training_target = preprocessing_utils.generate_dataset(train_original_data,
                                                   num_timesteps_input=num_timesteps_input,
                                                   num_timesteps_output=num_timesteps_output)
val_input, val_target = preprocessing_utils.generate_dataset(val_original_data,
                                         num_timesteps_input=num_timesteps_input,
                                         num_timesteps_output=num_timesteps_output)
test_input, test_target = preprocessing_utils.generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

adj_mat = preprocessing_utils.get_normalized_adj(A)
adj_mat = torch.from_numpy(adj_mat).float()

adj_mat = adj_mat.to(device)
training_input = training_input.to(device)
training_target = training_target.to(device)
val_input = val_input.to(device)
val_target = val_target.to(device)

Hyperparameter Optimization using Optuna

In [ ]:
def define_model(trial):
    return model.Stgcn_Model(nodes_num = adj_mat.shape[0], \
                             features_num = training_input.shape[3], \
                             input_timesteps = num_timesteps_input, \
                             num_output = num_timesteps_output)

def objective(trial):
    stgcn = define_model(trial).to(device)
    loss_criterion = nn.MSELoss()
    
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    optimizer = getattr(torch.optim, optimizer_name)(stgcn.parameters(), lr=lr)
    batch_size = trial.suggest_categorical("bs", [16, 32, 64])
    
    stgcn, training_loss, validation_loss = model_utils.train(stgcn, optimizer, lr, loss_criterion, epochs, patience, adj_mat,
                  training_input, training_target, val_input, val_target, batch_size)
    
    return validation_loss[np.argmin(validation_loss)]

In [ ]:
study = optuna.create_study()

study.optimize(objective, n_trials=30, timeout=3600)

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)